<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">


# DSI-13 Capstone Project: 
# Recommender Systems

_Authors: Davis Hong_

---

### Problem Statement:
A Recommender System refers to a system that is capable of predicting the future preference of a set of items for a user, and recommend somthing s/he likes (content based) or something preferred by other users with similar tastes (collaborative). In the past where brick and mortar is the norm, recommendations are usually made through word of mouth. However, with the internet, availability of large amount of data and fast computing power, recommendations not only can be done real-time but also fairly accurate. Recommendations are very common nowadays to the point, it has becoming 'in-your-face'. Businesses especially those relying heavily or solely on the internet for revenue, recommender systems will greatly boost the growth or even the very survival of the business.

There are many type of recommender systems but this project, we will focus on developing a recommender system to recommend movies to users based on a hybrid model;

<a id="two-classical-recommendation-methods"></a>
### Two classical recommendation methods

- **Collaborative Filtering**: _(similar people)_
    - If you like the same 5 movies as someone else, you'll likely enjoy other movies they like.
    - There are two main types: (a) Find users who are similar and recommend what they like (**user-based**), or (b) recommend items that are similar to already-liked items (**item-based**).
   

- **Content-Based Filtering** _(similar items)_
    - If you enjoy certain characteristics of movies (e.g. certain actors, genre, etc.), you'll enjoy other movies with those characteristics.
    - Note this can easily be done using machine learning methods! Each movie can be decomposed into features. Then, for each user we compute a model -- the target can be a binary classifier (e.g. "LIKE"/"DISLIKE") or regression (e.g. star rating).
<br>
The initial model is based on a simple non-machine learning model and the results will be compared with a machine learning based model. The machine learning model will be expected to produce better results.

### Data Source:
Data source: The 1M Movie Lens datasets will be used for the project.

Datasets can be found at https://grouplens.org/datasets/movielens/1m/

Data dictionary: http://files.grouplens.org/datasets/movielens/ml-1m-README.txt


### Potential challenges and obstacles:
As there are no metrics to measure whether the recommender engine is good or not thus some form of human interpretation of the results will be required to gauge the performance of models.

### Measuring of results.
Whenever possible, baseline score will be set and metrics such as MSE, RSME, etc will be used to measure the effectiveness of the model.

Bearing any unforeseen circumstances for eg, interruptions cause by COVID19, this project should be achievable as data is available. Data cleaning is expected but minimal.

### Let's get started...

### Load required libraries

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn import preprocessing
sns.set()
sns.set_style('darkgrid')
plt.style.use('fivethirtyeight')
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

## Load datasets
---

We'll be using the [MovieLens](https://grouplens.org/datasets/movielens/) dataset for building our recommendation engine. There are 3 datasets namely movies, ratings and users which will be used to build the recommender system. 

In [3]:
pwd

'P:\\GitHub\\General-Assembly-Projects\\Capstone'

In [14]:
movies = pd.read_csv('.\datasets\\movies.csv')
ratings = pd.read_csv('.\datasets\\ratings.csv')
users = pd.read_csv('.\datasets\\users.csv')

In [15]:
movies.head()

,movieid,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [16]:
ratings.head()

,userid,movieid,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [17]:
users.head()

,userid,gender,age,occupation,zipcode
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,2460
4,5,M,25,20,55455


In [21]:
# Per data dictionary, there are additional information on the numeric codes 
# used in the users dataset. 

#Create dictionary for age group.
data = {'age': [1, 18, 25, 35, 45, 50, 56], 
        'agegrp': ['Under 18', '18-24', '25-34', '35-44', '45-49', '50-55', '56+']} 

# Create DataFrame for age group
age_grp = pd.DataFrame(data)

# Create dictionary for occupation
data = {'code': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 
        'occupation': ['other', 'academic/educator', 'artist', 'cleric/admin',
                       'college/grad student', 'customer service',
                       'doctor/health care', 'executive/managerial', 'farmer',
                       'homemaker', 'K-12 student', 'lawyer', 'programmer',
                      'retired', 'sales/marketing', 'scientist',
                      'self-employed', 'technician/engineer',
                      'tradesman/craftsman', 'unemployed', 'writer']} 

# Create DataFrame for age group
occupation = pd.DataFrame(data)

## Drop unnecessary columns
---

We won't need the `timestamp` column from `ratings`, nor will we need the `genres` column from `movies`. Drop both columns in the cells below.

In [34]:
ratings.drop('timestamp', inplace = True, axis = 1)
movies.drop('genres', axis = 1, inplace = True)

## Merge `movies` and `ratings`
---

Use `pd.merge` to **inner join** `movies` with `ratings` on the `movieId` column.

In [35]:
df = pd.merge(ratings, movies, on = 'movieId')

## Create pivot table
---

Because we're creating an item-based collaborative recommender (where item in this case is our movies), we'll set up our pivot table as follows:
1. The `title` will be the index
2. The `userId` will be the column
3. The `rating` will be the value

**If we were building a user-based collaborative recommender, what would change about this pivot table?**

In [36]:
pivot = pd.pivot_table(df, index = 'title', columns='userId', values = 'rating')
pivot.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
'Hellboy': The Seeds of Creation (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Round Midnight (1986),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Salem's Lot (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Til There Was You (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Create sparse matrix
---

In a minute, we'll calculate the cosine similarity for each movie using the `pairwise_distances` function. Before that, we need to create a sparse matrix (datatype) using `scipy`'s `sparse` module like so:
```python
sparse.csr_matrix(pivot.fillna(0))
```

In [37]:
sparse_pivot = sparse.csr_matrix(pivot.fillna(0))

In [38]:
sparse_pivot

<9719x610 sparse matrix of type '<class 'numpy.float64'>'
	with 100832 stored elements in Compressed Sparse Row format>

## Calculate cosine similarity
---

`sklearn` has a built-in `pairwise_distances` function that we can use for our recommender. It will return a square matrix, comparing every movie with every other movie in the dataset.

```python
pairwise_distances(sparse_pivot, metric='cosine')
```

In [40]:
# Create a distance metrics similar to    
recommender = pairwise_distances(sparse_pivot, metric = 'cosine')

In [41]:
recommender_df = pd.DataFrame(recommender, columns = pivot.index, index = pivot.index)

In [42]:
recommender_df.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.0,0.858347,1.0,...,1.0,0.657945,0.456695,0.292893,1.0,1.0,0.860569,0.672673,1.0,1.0
'Hellboy': The Seeds of Creation (2004),1.0,0.000000,0.292893,1.000000,1.000000,1.0,1.000000,1.0,1.000000,1.0,...,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.0
'Round Midnight (1986),1.0,0.292893,0.000000,1.000000,1.000000,1.0,0.823223,1.0,1.000000,1.0,...,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.0
'Salem's Lot (2004),1.0,1.000000,1.000000,0.000000,0.142507,1.0,1.000000,1.0,1.000000,1.0,...,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.0
'Til There Was You (1997),1.0,1.000000,1.000000,0.142507,0.000000,1.0,1.000000,1.0,1.000000,1.0,...,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.0


## Create distances DataFrame
---

At this point, we essentially have a recommender. We'll load it into a `pandas` DataFrame for readability. 

You'll notice that each movie has a "distance" of 0 with itself (along the diagonal).

## Evaluate recommender performance
---

Now comes the fun part! Let's check out a few movies to see if the recommender aligns with our intuition. In the cell below we'll do the following:
1. Create a search term
2. Use that to find all titles matching the search query
3. For each title, we'll list off the following:
  1. The average rating
  2. The number of ratings
  3. The ten most similar movies

In [66]:
search = 'Narnia'

In [71]:
for title in movies.loc[movies['title'].str.contains(search), 'title']:
    print(title)
    print(f'Average rating: {pivot.loc[title, :].mean()}') # Average ratings from users
    print(f'Number of rating: {pivot.loc[title, :].count()}')
    print('')
    print('10 most similar movies')
    print('')
    print(recommender_df[title].sort_values()[1:11])
    print('')
    print('*' * 90)
    

Chronicles of Narnia: The Lion, the Witch and the Wardrobe, The (2005)
Average rating: 3.443548387096774
Number of rating: 62

10 most similar movies

title
Harry Potter and the Goblet of Fire (2005)                                                        0.359103
Pirates of the Caribbean: Dead Man's Chest (2006)                                                 0.371752
Charlie and the Chocolate Factory (2005)                                                          0.397919
Pirates of the Caribbean: The Curse of the Black Pearl (2003)                                     0.433360
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)    0.433687
Harry Potter and the Chamber of Secrets (2002)                                                    0.433769
Star Wars: Episode III - Revenge of the Sith (2005)                                               0.434553
Harry Potter and the Prisoner of Azkaban (2004)                                               